In [2]:
%pip install pgmpy

from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import numpy as np

# Define Bayesian Network structure
model = DiscreteBayesianNetwork([('Prize', 'Host'), ('Choice', 'Host')])

# CPDs for Prize and Choice (uniform distributions)
cpd_prize = TabularCPD('Prize', 3, [[1/3], [1/3], [1/3]])
cpd_choice = TabularCPD('Choice', 3, [[1/3], [1/3], [1/3]])

# CPD for Host based on Prize and Choice
host_vals = np.zeros((3, 9))
for i, p in enumerate([1, 2, 3]):
    for j, c in enumerate([1, 2, 3]):
        valid = [h for h in [1, 2, 3] if h != p and h != c]
        if len(valid) > 0:
            for h in valid:
                host_vals[h-1, 3*i + j] = 1 / len(valid)

cpd_host = TabularCPD('Host', 3, host_vals, ['Prize', 'Choice'], [3, 3])

# Add and verify model
model.add_cpds(cpd_prize, cpd_choice, cpd_host)
model.check_model()

# Perform inference
result = VariableElimination(model).query(['Prize'], evidence={'Choice': 0, 'Host': 2})

print("P(Prize | Choice=1, Host=3):")
print(result)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
P(Prize | Choice=1, Host=3):
+----------+--------------+
| Prize    |   phi(Prize) |
+==========+==============+
| Prize(0) |       0.3333 |
+----------+--------------+
| Prize(1) |       0.6667 |
+----------+--------------+
| Prize(2) |       0.0000 |
+----------+--------------+
